In [2]:
!uv add requests

Resolved 111 packages in 2.45s
Audited 107 packages in 0.10ms


In [1]:
import requests

In [4]:
repo_owner = 'evidentlyai'
repo_name = 'docs'
branch_name = 'main'

zip_url = f'https://github.com/{repo_owner}/{repo_name}/archive/refs/heads/{branch_name}.zip'
zip_response = requests.get(zip_url)

In [5]:
len(zip_response.content)

17545668

In [6]:
import io
import zipfile

zip_archive = zipfile.ZipFile(io.BytesIO(zip_response.content))

In [10]:
filenames = zip_archive.namelist()
filenames[20:30]

['docs-main/docs/library/report.mdx',
 'docs-main/docs/library/synthetic_data_api.mdx',
 'docs-main/docs/library/tags_metadata.mdx',
 'docs-main/docs/library/tests.mdx',
 'docs-main/docs/platform/',
 'docs-main/docs/platform/alerts.mdx',
 'docs-main/docs/platform/dashboard_add_panels.mdx',
 'docs-main/docs/platform/dashboard_add_panels_ui.mdx',
 'docs-main/docs/platform/dashboard_overview.mdx',
 'docs-main/docs/platform/dashboard_panel_types.mdx']

In [30]:
filename = 'docs-main/docs/platform/alerts.mdx'

mdx_file = zip_archive.open(filename)
mdx_content = mdx_file.read().decode('utf8')

In [22]:
!uv add python-frontmatter

Resolved 112 packages in 399ms
Installed 1 package in 126ms
 + python-frontmatter==1.1.0


In [23]:
import frontmatter

post = frontmatter.loads(mdx_content)

<Check>
  Built-in alerting is a Pro feature available in the **Evidently Cloud** and **Evidently En


In [25]:
post.metadata

{'title': 'Alerts', 'description': 'How to set up alerts.'}

In [28]:
print(post.content[:100])

<Check>
  Built-in alerting is a Pro feature available in the **Evidently Cloud** and **Evidently En


In [39]:
_, filename_corrected = filename.split('/', maxsplit=1)
print(filename_corrected)

docs/platform/alerts.mdx


In [43]:
doc = {
    'content': post.content,
    'title': post.metadata.get('title'),
    'description': post.metadata.get('description'),
    'filename': filename_corrected
}

In [45]:
documents = []
with zipfile.ZipFile(io.BytesIO(zip_response.content)) as zip_ref:
    for file_path in zip_ref.namelist():
        if not file_path.endswith(('.md', '.mdx')):
            continue
        with zip_ref.open(file_path) as file:
            content = file.read().decode('utf-8')
            post = frontmatter.loads(content)
            doc = {
                'content': post.content,
                'title': post.metadata.get('title'),
                'description': post.metadata.get('description'),
                'filename': file_path.split('/', 1)[-1]
            }
            documents.append(doc)

In [46]:
len(documents)

95

In [47]:
!uv add gitsource

Resolved 113 packages in 1.49s
Installed 1 package in 83ms
 + gitsource==0.0.4


In [48]:
from gitsource import GithubRepositoryDataReader

reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)

files = reader.read()

print(f"Loaded {len(files)} documents")


Loaded 95 documents


In [52]:
md_file = files[10]

In [56]:
documents = [f.parse() for f in files]

In [57]:
len(documents)

95

In [58]:
documents[10]

{'title': 'Output formats',
 'description': 'How to export the evaluation results.',
 'content': 'You can view or export Reports in multiple formats.\n\n**Pre-requisites**:\n\n* You know how to [generate Reports](/docs/library/report).\n\n## Log to Workspace\n\nYou can save the computed Report in Evidently Cloud or your local workspace.\n\n```python\nws.add_run(project.id, my_eval, include_data=False)\n```\n\n<Info>\n  **Uploading evals**. Check Quickstart examples [for ML](/quickstart_ml) or [for LLM](/quickstart_llm) for a full workflow.\n</Info>\n\n## View in Jupyter notebook\n\nYou can directly render the visual summary of evaluation results in interactive Python environments like Jupyter notebook or Colab.\n\nAfter running the Report, simply call the resulting Python object:\n\n```python\nmy_report\n```\n\nThis will render the HTML object directly in the notebook cell.\n\n## HTML\n\nYou can also save this interactive visual Report as an HTML file to open in a browser:\n\n```python